Spam Filter Classification Model

In [1]:
#importing all libraries required

In [2]:
#core libraries
import pandas as pd
import numpy as np

#sklearn processing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold,cross_val_score

#sklearn classification algorithms
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

#sklearn classification model evaluation functions
from sklearn import metrics

Data Collection

In [3]:
#Since the data is already processed , we just store the processed data into a csv file.
rf=pd.read_csv('spambase.data',header=None,delimiter=',')
rf.columns=['word_freq_make','word_freq_address','word_freq_all','word_freq_3d','word_freq_our','word_freq_over','word_freq_remove','word_freq_internet',
'word_freq_order','word_freq_mail','word_freq_receive','word_freq_will','word_freq_people','word_freq_report','word_freq_addresses',
'word_freq_free','word_freq_business','word_freq_email','word_freq_you','word_freq_credit','word_freq_your','word_freq_font','word_freq_000',
'word_freq_money','word_freq_hp','word_freq_hpl','word_freq_george','word_freq_650','word_freq_lab','word_freq_labs','word_freq_telnet',
'word_freq_857','word_freq_data','word_freq_415','word_freq_85','word_freq_technology','word_freq_1999','word_freq_parts',
'word_freq_pm','word_freq_direct','word_freq_cs','word_freq_meeting','word_freq_original','word_freq_project','word_freq_re','word_freq_edu',
'word_freq_table','word_freq_conference','char_freq_;','char_freq_(','char_freq_[','char_freq_!','char_freq_$','char_freq_#','capital_run_length_average',
'capital_run_length_longest','capital_run_length_total','class']
rf.to_csv('SpamBase.csv',index=None)

Data Preparation

In [4]:
#Load the data
data=pd.read_csv("SpamBase.csv")

In [5]:
#Inspect Data
#Inspecting the column names
data.columns

Index(['word_freq_make', 'word_freq_address', 'word_freq_all', 'word_freq_3d',
       'word_freq_our', 'word_freq_over', 'word_freq_remove',
       'word_freq_internet', 'word_freq_order', 'word_freq_mail',
       'word_freq_receive', 'word_freq_will', 'word_freq_people',
       'word_freq_report', 'word_freq_addresses', 'word_freq_free',
       'word_freq_business', 'word_freq_email', 'word_freq_you',
       'word_freq_credit', 'word_freq_your', 'word_freq_font', 'word_freq_000',
       'word_freq_money', 'word_freq_hp', 'word_freq_hpl', 'word_freq_george',
       'word_freq_650', 'word_freq_lab', 'word_freq_labs', 'word_freq_telnet',
       'word_freq_857', 'word_freq_data', 'word_freq_415', 'word_freq_85',
       'word_freq_technology', 'word_freq_1999', 'word_freq_parts',
       'word_freq_pm', 'word_freq_direct', 'word_freq_cs', 'word_freq_meeting',
       'word_freq_original', 'word_freq_project', 'word_freq_re',
       'word_freq_edu', 'word_freq_table', 'word_freq_conference',


In [6]:
#inspecting the number of rows and columns
data.shape

(4601, 58)

In [7]:
#Having a look at the data
data.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,class
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [8]:
#Checking the data types of each feature
data.dtypes

word_freq_make                float64
word_freq_address             float64
word_freq_all                 float64
word_freq_3d                  float64
word_freq_our                 float64
word_freq_over                float64
word_freq_remove              float64
word_freq_internet            float64
word_freq_order               float64
word_freq_mail                float64
word_freq_receive             float64
word_freq_will                float64
word_freq_people              float64
word_freq_report              float64
word_freq_addresses           float64
word_freq_free                float64
word_freq_business            float64
word_freq_email               float64
word_freq_you                 float64
word_freq_credit              float64
word_freq_your                float64
word_freq_font                float64
word_freq_000                 float64
word_freq_money               float64
word_freq_hp                  float64
word_freq_hpl                 float64
word_freq_ge

In [9]:
#Inspecting the range and spread of values of each feature
data.describe()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,class
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.104553,0.213015,0.280656,0.065425,0.312223,0.095901,0.114208,0.105295,0.090067,0.239413,...,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285,0.394045
std,0.305358,1.290575,0.504143,1.395151,0.672513,0.273824,0.391441,0.401071,0.278616,0.644755,...,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.276000,15.000000,95.000000,0.000000
75%,0.000000,0.000000,0.420000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.160000,...,0.000000,0.188000,0.000000,0.315000,0.052000,0.000000,3.706000,43.000000,266.000000,1.000000
max,4.540000,14.280000,5.100000,42.810000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,...,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000,1.000000


In [10]:
#checking for null values
data.columns.isnull()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False])

Data Transformation

In [11]:
#x is the data to be trained on , y is the class values to test on
x=data.iloc[:,:-1]
y=data['class']

In [12]:
#Having a look at the training data
x.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.0,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191


In [13]:
#Inspecting the shape of x and y
print(x.shape,y.shape)

(4601, 57) (4601,)


In [14]:
#Normalizing the data using Min Max Scaler which will rescale the data so that all values will be in the range 0 and 1
scaler=MinMaxScaler()
#fit estimates the min and max values in the training data
scaler.fit(x)
#transform normalizes the data
x_scaled=scaler.transform(x)
x_scaled=pd.DataFrame(x_scaled)

Classification Models

Naive Bayes Classifier

In [15]:
classifier=GaussianNB()
scores=cross_val_score(classifier,x_scaled,y,cv=10)
print("Accuracy",np.mean(scores),"Standard Deviation",scores.std())

Accuracy 0.8187296048288222 Standard Deviation 0.07743520649019187


SVM Classifier

In [16]:
classifier=SVC(kernel='rbf',gamma='auto')
scores=cross_val_score(classifier,x_scaled,y,cv=10)
print("Accuracy",np.mean(scores),"Standard Deviation",scores.std())

Accuracy 0.8037465811562765 Standard Deviation 0.028343158792402096


Decision Tree Classifier

In [17]:
classifier=DecisionTreeClassifier(criterion="entropy",random_state=0)
scores=cross_val_score(classifier,x_scaled,y,cv=10)
print("Accuracy",np.mean(scores),"Standard Deviation",scores.std())

Accuracy 0.9126204847684619 Standard Deviation 0.03333432664822017


Bagging Classifier

In [18]:
classifier=BaggingClassifier(base_estimator=SVC())
scores=cross_val_score(classifier,x_scaled,y,cv=10)
print("Accuracy",np.mean(scores),"Standard Deviation",scores.std())

Accuracy 0.9278416485900218 Standard Deviation 0.025217702820275856


K-Nearest Neighbor Classifier

In [19]:
#estimating optimal value of k
for i in range(1,10):
    classifier=KNeighborsClassifier(n_neighbors=i,weights='distance')
    scores=cross_val_score(classifier,x_scaled,y,cv=10)
    print(i," ","Accuracy",np.mean(scores),"Standard Deviation",scores.std())

1   Accuracy 0.8856818824860889 Standard Deviation 0.03729749539243028
2   Accuracy 0.8863340563991324 Standard Deviation 0.03663022381754964
3   Accuracy 0.8876407620484768 Standard Deviation 0.03432656816907546
4   Accuracy 0.8911166650947846 Standard Deviation 0.03166303411074864
5   Accuracy 0.8895944543996981 Standard Deviation 0.03262755757031515
6   Accuracy 0.894156370838442 Standard Deviation 0.030563466743595655
7   Accuracy 0.8917655380552674 Standard Deviation 0.03206876907299416
8   Accuracy 0.8950264076204849 Standard Deviation 0.030714880083931823
9   Accuracy 0.8913331132698292 Standard Deviation 0.03673272837881641


In [20]:
#optimal value is 7 because we have even number of classes to be predicted ,inorder to avoid ties we need to chose a odd value of k
k_classifier=KNeighborsClassifier(n_neighbors=7,weights='distance')
scores=cross_val_score(classifier,x_scaled,y,cv=10)
print("Accuracy",np.mean(scores),"Standard Deviation",scores.std())

Accuracy 0.8913331132698292 Standard Deviation 0.03673272837881641


Random Forest Classifier

In [21]:
classifier=RandomForestClassifier(criterion="entropy",random_state=0)
scores=cross_val_score(classifier,x_scaled,y,cv=10)
print("Accuracy",np.mean(scores),"Standard Deviation",scores.std())
#The best performing model is Random Forest Classifier due to its high accuracy compared to other models

Accuracy 0.9391412807695936 Standard Deviation 0.035029700844193566


Evaluating the Results

Best Performing Model-Random Forest Classifier

In [22]:
#k fold Cross validation
kcv=KFold(n_splits=10,shuffle=True)
model=RandomForestClassifier(criterion="entropy",random_state=0)

In [23]:
sumE=0
#number of folds k=10
k=10
i_list=[]
false_positive_list=[]
false_negative_list=[]
error_rate_list=[]
accuracy_list=[]
for i in range(1,k+1):
    result=next(kcv.split(x_scaled),None)

    #splitting into train and test sets
    x_train=x_scaled.iloc[result[0]]
    x_test=x_scaled.iloc[result[1]]
    y_train=y.iloc[result[0]]
    y_test=y.iloc[result[1]]

    #fitting the model
    model.fit(x_train,y_train)
    #predicting the values
    y_pred=model.predict(x_test)

    conf_matrix=metrics.confusion_matrix(y_test,y_pred)
    #false positive is at [0][1] of 2X2 confusion matrix
    fp=conf_matrix[0][1]
    #false negative is at [1][0] of 2X2 confusion matrix
    fn=conf_matrix[1][0]
    #true negative is at [0][0] of 2X2 confusion matrix
    tn=conf_matrix[0][0]
    #true positive is at [1][1] of 2X2 confusion matrix
    tp=conf_matrix[1][1]
    #false positive rate
    fpr=(fp/(fp+tn))
    #false negative rate
    fnr=(fn/(fn+tp))
    #overall error rate
    err=((fp+fn)/(tp+tn+fp+fn))
    #sum of all error rates
    sumE=sumE+err

    i_list.append(i)
    false_positive_list.append(fpr)
    false_negative_list.append(fnr)
    error_rate_list.append(err)

    #calculating accuracy
    accuracy_list.append(metrics.accuracy_score(y_test,y_pred))

In [24]:
i_list.append("Average")
false_positive_list.append(np.mean(false_positive_list))
false_negative_list.append(np.mean(false_negative_list))
error_rate_list.append(np.mean(error_rate_list))
accuracy_list.append(np.mean(accuracy_list))

from tabulate import tabulate
df=pd.DataFrame({"Fold Number":i_list,"False Positive Rate":false_positive_list,"False Negative Rate":false_negative_list,"Error Rate":error_rate_list,"Accuracy":accuracy_list})
df.set_index(["Fold Number"],inplace=True)
print(tabulate(df,headers='keys',tablefmt='psql'))

+---------------+-----------------------+-----------------------+--------------+------------+
| Fold Number   |   False Positive Rate |   False Negative Rate |   Error Rate |   Accuracy |
|---------------+-----------------------+-----------------------+--------------+------------|
| 1             |             0.0262172 |             0.0824742 |    0.0498915 |   0.950108 |
| 2             |             0.0149813 |             0.0515464 |    0.0303688 |   0.969631 |
| 3             |             0.018315  |             0.0478723 |    0.0303688 |   0.969631 |
| 4             |             0.0206186 |             0.0529412 |    0.032538  |   0.967462 |
| 5             |             0.0252708 |             0.0380435 |    0.0303688 |   0.969631 |
| 6             |             0.0316901 |             0.0564972 |    0.0412148 |   0.958785 |
| 7             |             0.0262172 |             0.0670103 |    0.0433839 |   0.956616 |
| 8             |             0.0173611 |             0.0924